<a href="https://colab.research.google.com/github/pruhnav/LoadForecasting/blob/main/Hybrid_Attention_BiLSTM_48_12_REALDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler


In [2]:

# Load real dataset from Excel
df = pd.read_excel("day_ahead_energy_prices.xlsx")

# Check column names
print("Available columns:", df.columns)

# Normalize the 'Price' column
scaler = MinMaxScaler()
data = scaler.fit_transform(df['Price'].values.reshape(-1, 1)).flatten()

# Create sliding window function
def create_sliding_window(data, input_len=48, output_len=12):
    X, y = [], []
    for i in range(len(data) - input_len - output_len + 1):
        X.append(data[i:i+input_len])
        y.append(data[i+input_len:i+input_len+output_len])
    return np.array(X), np.array(y)

# Generate input-output sequences
X, y = create_sliding_window(data, input_len=48, output_len=12)
X = X.reshape((X.shape[0], X.shape[1], 1))  # reshape for LSTM

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Available columns: Index(['Date', 'Price'], dtype='object')


In [3]:

def build_bilstm_attention_mlp(input_shape, output_dim):
    inputs = Input(shape=input_shape)
    x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x_att = MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
    x_att = LayerNormalization(epsilon=1e-6)(x + x_att)
    x = GlobalAveragePooling1D()(x_att)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(output_dim)(x)
    return Model(inputs, outputs)

model = build_bilstm_attention_mlp((48, 1), 12)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 48, 128)   │     33,792 │ input_layer[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 48, 128)   │     98,816 │ bidirectional[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 48, 128)   │     66,048 │ bidirectional_1[… │
│ (MultiHeadAttentio… │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 48, 128)   │          0 │ bidirectional_1[… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 48, 128)   │        256 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     16,512 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 12)        │      1,548 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 216,972 (847.55 KB)

 Trainable params: 216,972 (847.55 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:

history = model.fit(X_train, y_train, validation_split=0.1, epochs=3, batch_size=32, verbose=1)



Epoch 1/3
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 225s 143ms/step - loss: 0.0011 - mae: 0.0189 - val_loss: 7.9953e-04 - val_mae: 0.0163
Epoch 2/3
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 262s 143ms/step - loss: 0.0011 - mae: 0.0190 - val_loss: 8.4736e-04 - val_mae: 0.0195
Epoch 3/3
1577/1577 ━━━━━━━━━━━━━━━━━━━━ 257s 139ms/step - loss: 0.0010 - mae: 0.0184 - val_loss: 9.4948e-04 - val_mae: 0.0192


In [10]:

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
smape = 100 * np.mean(2 * np.abs(y_pred - y_test) / (np.abs(y_pred) + np.abs(y_test)))
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"SMAPE: {smape:.4f}")
print(f"R²: {r2:.4f}")


438/438 ━━━━━━━━━━━━━━━━━━━━ 21s 45ms/step
MSE: 0.0009
RMSE: 0.0304
MAE: 0.0190
SMAPE: 25.2036
R²: 0.7754
